
# 소리→빛 번역 AI 모델 개발 노트북

이 노트북은 **가정 내 생활 소리 분류**를 위해 전이학습(EfficientNetB0)과 **로그-멜 스펙트로그램**을 사용합니다.  
흐름: **데이터 준비 → 전처리(슬라이딩 윈도우) → 학습 → 파일/실시간 추론**

- 분류 대상(6종): 화재경보, 초인종/노크/도어락, 전화벨/알림, 가전제품 완료음, 아이 울음/강아지 짖음, 발소리/충격음  
- 색상 매핑: 빨강/주황/노랑/초록/파랑/보라


## 📦 의존성 설치 (필요 시 실행)

In [ ]:

# (Colab/새 환경이라면 실행)
# !pip install numpy scipy librosa==0.10.2.post1 soundfile sounddevice matplotlib tensorflow==2.15.* tensorflow-io==0.36.*


## ⚙️ 임포트 & 공통 설정

In [3]:
import os, glob, json, time, queue, math
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from pathlib import Path
from scipy.ndimage import zoom
import tensorflow as tf
import unicodedata

# ==== 공통 하이퍼파라미터 ====
SR = 16000               # 샘플레이트
WIN_SEC = 1.0            # 윈도우 길이 (초)
HOP_SEC = 0.5            # 슬라이딩 스텝 (초)
N_MELS = 64              # 멜 밴드 수
FMIN, FMAX = 50, 8000    # 주파수 대역
SPEC_SHAPE = (128, 128)  # 스펙트로그램 리사이즈
DATA_DIR = ".."
CACHE_DIR = "../cache"
MODEL_DIR = "../models"
Path(CACHE_DIR).mkdir(exist_ok=True)
Path(MODEL_DIR).mkdir(exist_ok=True)

# ==================== 한글 클래스명으로 업데이트 ====================
# eda_and_model_selection_v2.ipynb와 동일하게 설정
CLASSES = [
    "가전제품_종료",
    "개",
    "문_노크",
    "문_도어락",
    "문_초인종",
    "배경소음",
    "소음_가구",
    "소음_드럼",
    "소음_문소리",
    "소음_아이발소리",
    "소음_어른발소리",
    "소음_청소기",
    "소음_통돌이",
    "아기_울음",
    "안전_낙상",
    "안전_화재_경보",
    "정적",
    "타이머_알람",
    "폰_링톤",
    "폰_알림",
    "폰_전화",
]

# ==================== 색상 매핑 (카테고리별) ====================
# 대표 그룹별 색상 지정
# | 대표 그룹 | 하위 폴더 예시             | 색상    |
# | ----- | -------------------- | ----- |
# | 안전 관련 | 안전_낙상, 안전_화재_경보      | 🔴 빨강 |
# | 문 관련  | 문_노크, 문_도어락, 문_초인종   | 🟠 주황 |
# | 알림 관련 | 타이머_알람, 폰_알림, 폰_전화   | 🟡 노랑 |
# | 가전 관련 | 가전제품_종료, 소음_청소기      | 🟢 초록 |
# | 사람·동물 | 아기_울음, 개                | 🔵 파랑 |
# | 일반 소음 | 소음_가구, 소음_드럼, 소음_발소리 | 🟣 보라 |

CLASS_TO_COLOR = {
    # 🔴 안전 관련
    "안전_낙상": "RED",
    "안전_화재_경보": "RED",
    
    # 🟠 문 관련
    "문_노크": "ORANGE",
    "문_도어락": "ORANGE",
    "문_초인종": "ORANGE",
    
    # 🟡 알림 관련
    "타이머_알람": "YELLOW",
    "폰_링톤": "YELLOW",
    "폰_알림": "YELLOW",
    "폰_전화": "YELLOW",
    
    # 🟢 가전 관련
    "가전제품_종료": "GREEN",
    "소음_청소기": "GREEN",
    
    # 🔵 사람·동물
    "아기_울음": "BLUE",
    "개": "BLUE",
    
    # 🟣 일반 소음
    "소음_가구": "PURPLE",
    "소음_드럼": "PURPLE",
    "소음_문소리": "PURPLE",
    "소음_아이발소리": "PURPLE",
    "소음_어른발소리": "PURPLE",
    "소음_통돌이": "PURPLE",
    
    # 그 외
    "배경소음": "GRAY",
    "정적": "WHITE",
}

print("TensorFlow:", tf.__version__)
print(f"📚 로드된 클래스: {len(CLASSES)}개")
print(f"   {', '.join(CLASSES[:5])}...등등")

TensorFlow: 2.19.0
📚 로드된 클래스: 21개
   가전제품_종료, 개, 문_노크, 문_도어락, 문_초인종...등등


## 🎼 유틸리티: 슬라이딩 윈도우 & 로그-멜 스펙트로그램

In [6]:

def wav_to_windows(path, sr=SR, win_sec=WIN_SEC, hop_sec=HOP_SEC):
    y, sr = librosa.load(path, sr=sr, mono=True)
    win = int(sr * win_sec)
    hop = int(sr * hop_sec)
    if len(y) < win:
        y = np.pad(y, (0, win - len(y)))
    frames = []
    for start in range(0, len(y) - win + 1, hop):
        frames.append(y[start:start+win])
    return frames

def frames_to_logmels(frames, sr=SR, n_mels=N_MELS, fmin=FMIN, fmax=FMAX, shape=SPEC_SHAPE):
    specs = []
    for fr in frames:
        mel = librosa.feature.melspectrogram(
            y=fr, sr=sr, n_fft=1024, hop_length=256,
            n_mels=n_mels, fmin=fmin, fmax=fmax, power=2.0
        )
        logmel = librosa.power_to_db(mel, ref=np.max)
        zy = shape[0] / logmel.shape[0]
        zx = shape[1] / logmel.shape[1]
        img = zoom(logmel, (zy, zx), order=1)[..., None]  # (H, W, 1)
        specs.append(img.astype(np.float32))
    return np.array(specs, dtype=np.float32)


## 🧹 전처리: 폴더→슬라이스→스펙트로그램 → NPY 저장

## 🧹 전처리: 데이터 로딩 (Train + Test 폴더 통합)

In [17]:

# 1. 데이터 분포 확인
print("=" * 60)
print("📊 현재 데이터 분포 확인")
print("=" * 60)

for split in ["train", "val"]:
    npy_path = f"{CACHE_DIR}/X_{split}.npy"
    y_path = f"{CACHE_DIR}/y_{split}.npy"
    
    if os.path.exists(npy_path):
        X = np.load(npy_path)
        y = np.load(y_path)
        
        print(f"\n{split.upper()} 세트:")
        print(f"  전체 샘플: {len(y)}")
        for i, class_name in enumerate(CLASSES):
            count = (y == i).sum()
            percentage = (count / len(y)) * 100
            print(f"  [{i}] {class_name:20s}: {count:5d} ({percentage:5.1f}%)")

print("\n" + "=" * 60)
print("⚠️  데이터 불균형이 심하면 모델이 다수 클래스에 편향됩니다!")
print("=" * 60)


📊 현재 데이터 분포 확인

TRAIN 세트:
  전체 샘플: 16463
  [0] 가전제품_종료             :   239 (  1.5%)
  [1] 개                   :   369 (  2.2%)
  [2] 문_노크                :   123 (  0.7%)
  [3] 문_도어락               :    56 (  0.3%)
  [4] 문_초인종               :   262 (  1.6%)
  [5] 배경소음                :  1963 ( 11.9%)
  [6] 소음_가구               :  1221 (  7.4%)
  [7] 소음_드럼               :  1753 ( 10.6%)
  [8] 소음_문소리              :  1249 (  7.6%)
  [9] 소음_아이발소리            :  2096 ( 12.7%)
  [10] 소음_어른발소리            :  1635 (  9.9%)
  [11] 소음_청소기              :   986 (  6.0%)
  [12] 소음_통돌이              :  1668 ( 10.1%)
  [13] 아기_울음               :  1090 (  6.6%)
  [14] 안전_낙상               :   510 (  3.1%)
  [15] 안전_화재_경보            :   125 (  0.8%)
  [16] 정적                  :   153 (  0.9%)
  [17] 타이머_알람              :    87 (  0.5%)
  [18] 폰_링톤                :   134 (  0.8%)
  [19] 폰_알림                :   308 (  1.9%)
  [20] 폰_전화                :   436 (  2.6%)

VAL 세트:
  전체 샘플: 4116
  [0] 가전제품_종료         

In [19]:
# 2. 데이터 균형 맞추기 (적응적 샘플링)
def balance_dataset_adaptive(X, y, min_samples_threshold=50, max_samples_target=500):
    """
    데이터셋의 클래스 균형을 적응적으로 맞춥니다.
    - 너무 적은 클래스(< min_samples_threshold): 그대로 유지
    - 많은 클래스: max_samples_target으로 다운샘플링
    
    Args:
        X: 특성 데이터
        y: 레이블
        min_samples_threshold: 이 값보다 적은 클래스는 그대로 유지
        max_samples_target: 많은 클래스의 최대 목표 샘플 수
    """
    unique_classes = np.unique(y)
    
    # 각 클래스의 샘플 수 확인
    class_counts = {cls: (y == cls).sum() for cls in unique_classes}
    
    print(f"\n🎯 적응적 균형 조정 (최소: {min_samples_threshold}, 최대: {max_samples_target})")
    print("-" * 80)
    
    balanced_X = []
    balanced_y = []
    
    for cls in unique_classes:
        cls_indices = np.where(y == cls)[0]
        original_count = len(cls_indices)
        
        # 샘플링 전략 결정
        if original_count <= min_samples_threshold:
            # 적은 클래스: 그대로 유지
            selected_indices = cls_indices
            strategy = "유지"
        elif original_count > max_samples_target:
            # 많은 클래스: 다운샘플링
            selected_indices = np.random.choice(cls_indices, max_samples_target, replace=False)
            strategy = "다운샘플링"
        else:
            # 중간 클래스: 그대로 유지
            selected_indices = cls_indices
            strategy = "유지"
        
        balanced_X.append(X[selected_indices])
        balanced_y.append(y[selected_indices])
        
        print(f"  {CLASSES[cls]:20s}: {original_count:5d} → {len(selected_indices):5d} ({strategy})")
    
    balanced_X = np.concatenate(balanced_X, axis=0)
    balanced_y = np.concatenate(balanced_y, axis=0)
    
    # 셔플
    shuffle_idx = np.random.permutation(len(balanced_y))
    balanced_X = balanced_X[shuffle_idx]
    balanced_y = balanced_y[shuffle_idx]
    
    print(f"\n✅ 총 샘플: {len(y)} → {len(balanced_y)}")
    return balanced_X, balanced_y

# Train 데이터 균형 맞추기
print("\n" + "=" * 60)
print("⚖️  데이터 균형 조정 중...")
print("=" * 60)

# NPY 파일에서 로드
X_train = np.load(f"{CACHE_DIR}/X_train.npy")
y_train = np.load(f"{CACHE_DIR}/y_train.npy")

print("\n📥 Train 데이터 적응적 균형 조정:")
X_train_balanced, y_train_balanced = balance_dataset_adaptive(
    X_train, y_train, 
    min_samples_threshold=50,  # 50개 미만은 그대로 유지
    max_samples_target=500     # 500개로 제한
)

# Test 데이터는 균형 조정하지 않음 (실제 분포 유지)
X_test = np.load(f"{CACHE_DIR}/X_test.npy")
y_test = np.load(f"{CACHE_DIR}/y_test.npy")

# Val 데이터도 균형 조정하지 않음
try:
    X_val = np.load(f"{CACHE_DIR}/X_val.npy")
    y_val = np.load(f"{CACHE_DIR}/y_val.npy")
    print(f"\n📥 Validation 데이터: {X_val.shape} (실제 분포 유지)")
except:
    X_val, y_val = None, None
    print(f"\n📥 Validation 데이터: 없음")

# 균형 맞춘 Train 데이터만 저장
np.save(f"{CACHE_DIR}/X_train_balanced.npy", X_train_balanced)
np.save(f"{CACHE_DIR}/y_train_balanced.npy", y_train_balanced)

print(f"\n💾 균형 맞춘 Train 데이터 저장 완료!")
print(f"   Train (원본):     {X_train.shape}")
print(f"   Train (균형):     {X_train_balanced.shape}")
print(f"   Test (원본 유지): {X_test.shape}")
if X_val is not None:
    print(f"   Val (원본 유지):  {X_val.shape}")

# 균형 조정 후 클래스별 분포 확인
print("\n📊 균형 조정 후 Train 분포:")
print("-" * 80)
for ci, cname in enumerate(CLASSES):
    count = (y_train_balanced == ci).sum()
    percentage = (count / len(y_train_balanced)) * 100 if len(y_train_balanced) > 0 else 0
    print(f"   [{ci:2d}] {cname:20s}: {count:6d} ({percentage:5.1f}%)")


⚖️  데이터 균형 조정 중...

📥 Train 데이터 적응적 균형 조정:

🎯 적응적 균형 조정 (최소: 50, 최대: 500)
--------------------------------------------------------------------------------
  가전제품_종료             :   137 →   137 (유지)
  개                   :   266 →   266 (유지)
  문_노크                :    69 →    69 (유지)
  문_도어락               :    44 →    44 (유지)
  문_초인종               :   207 →   207 (유지)
  배경소음                :  1564 →   500 (다운샘플링)
  소음_가구               :   809 →   500 (다운샘플링)
  소음_드럼               :  1396 →   500 (다운샘플링)
  소음_문소리              :   939 →   500 (다운샘플링)
  소음_아이발소리            :  1635 →   500 (다운샘플링)
  소음_어른발소리            :  1204 →   500 (다운샘플링)
  소음_청소기              :   748 →   500 (다운샘플링)
  소음_통돌이              :  1407 →   500 (다운샘플링)
  아기_울음               :   876 →   500 (다운샘플링)
  안전_낙상               :   386 →   386 (유지)
  안전_화재_경보            :   113 →   113 (유지)
  정적                  :   112 →   112 (유지)
  타이머_알람              :    67 →    67 (유지)
  폰_링톤                :    65 →    65 (유지)


In [18]:
def build_split_separate(data_dir):
    """
    Train과 Test 폴더를 분리하여 데이터 로드
    - train 폴더: 학습용 (80% 학습, 20% 내부 테스트)
    - test 폴더: 검증용
    
    Args:
        data_dir: 상위 데이터 폴더 경로
    
    Returns:
        train_data: (X_train, y_train, X_test, y_test)
        val_data: (X_val, y_val)
    """
    # 한글 클래스명 정규화 문제 처리
    norm_classes = set(unicodedata.normalize('NFC', c) for c in CLASSES)
    
    print("=" * 80)
    print("📂 데이터 로드 중... (Train/Test 분리)")
    print("=" * 80)
    
    def load_split_data(split_name):
        """개별 폴더 데이터 로드"""
        X, y = [], []
        split_dir = os.path.join(data_dir, split_name)
        
        if not os.path.exists(split_dir):
            print(f"⚠️  폴더 없음: {split_dir}")
            return None, None
        
        print(f"\n📁 {split_name.upper()} 폴더 처리 중...")
        split_count = 0
        
        for class_name in os.listdir(split_dir):
            # 한글 정규화
            n_class = unicodedata.normalize('NFC', class_name)
            if n_class not in norm_classes:
                continue
            
            # 클래스 인덱스 찾기
            class_idx = CLASSES.index(n_class)
            class_dir = os.path.join(split_dir, class_name)
            
            if not os.path.isdir(class_dir):
                continue
            
            # 해당 클래스의 모든 오디오 파일 로드
            audio_files = glob.glob(os.path.join(class_dir, "*.wav")) + \
                         glob.glob(os.path.join(class_dir, "*.m4a")) + \
                         glob.glob(os.path.join(class_dir, "*.flac"))
            
            file_count = 0
            for audio_path in audio_files:
                try:
                    frames = wav_to_windows(audio_path)
                    specs = frames_to_logmels(frames)
                    labels = np.full((specs.shape[0],), class_idx, dtype=np.int64)
                    X.append(specs)
                    y.append(labels)
                    file_count += 1
                except Exception as e:
                    print(f"  ⚠️  {audio_path}: {str(e)}")
            
            if file_count > 0:
                print(f"   [{class_idx:2d}] {n_class:20s}: {file_count:3d} 파일 ✓")
                split_count += file_count
        
        if not X:
            print(f"❌ {split_name} 폴더에서 로드된 데이터 없음!")
            return None, None
        
        X = np.concatenate(X, axis=0)
        y = np.concatenate(y, axis=0)
        
        print(f"   {split_name.upper()} 소계: {split_count} 파일, {len(y)} 샘플")
        return X, y
    
    # Train 데이터 로드
    X_train_full, y_train_full = load_split_data("train")
    
    # Test 데이터 로드 (검증용)
    X_val, y_val = load_split_data("test")
    
    if X_train_full is None:
        print("❌ Train 데이터를 로드할 수 없습니다!")
        return None, None
    
    # Train 데이터를 80/20으로 분할
    split_idx = int(0.8 * len(y_train_full))
    indices = np.random.permutation(len(y_train_full))
    
    train_indices = indices[:split_idx]
    test_indices = indices[split_idx:]
    
    X_train = X_train_full[train_indices]
    y_train = y_train_full[train_indices]
    X_test = X_train_full[test_indices]
    y_test = y_train_full[test_indices]
    
    print("\n" + "=" * 80)
    print("✅ 데이터 로드 완료!")
    print(f"   Train:    {X_train.shape}")
    print(f"   Test:     {X_test.shape}")
    if X_val is not None:
        print(f"   Val:      {X_val.shape}")
    else:
        print("   Val:      없음 (test 폴더 없음)")
    print("=" * 80)
    
    return (X_train, y_train, X_test, y_test), (X_val, y_val)

# 데이터 로드 (train/test 분리)
print("\n🚀 데이터 로드 시작...\n")
train_data, val_data = build_split_separate(DATA_DIR)

if train_data is not None:
    X_train, y_train, X_test, y_test = train_data
    X_val, y_val = val_data if val_data[0] is not None else (None, None)
    
    # Train 클래스별 분포 확인
    print("\n📊 TRAIN 클래스별 샘플 수:")
    print("-" * 80)
    for ci, cname in enumerate(CLASSES):
        count = (y_train == ci).sum()
        percentage = (count / len(y_train)) * 100 if len(y_train) > 0 else 0
        print(f"   [{ci:2d}] {cname:20s}: {count:6d} ({percentage:5.1f}%)")
    
    # Test 클래스별 분포 확인
    print("\n📊 TEST (from train) 클래스별 샘플 수:")
    print("-" * 80)
    for ci, cname in enumerate(CLASSES):
        count = (y_test == ci).sum()
        percentage = (count / len(y_test)) * 100 if len(y_test) > 0 else 0
        print(f"   [{ci:2d}] {cname:20s}: {count:6d} ({percentage:5.1f}%)")
    
    # Validation 클래스별 분포 확인
    if X_val is not None:
        print("\n📊 VAL (test folder) 클래스별 샘플 수:")
        print("-" * 80)
        for ci, cname in enumerate(CLASSES):
            count = (y_val == ci).sum()
            percentage = (count / len(y_val)) * 100 if len(y_val) > 0 else 0
            print(f"   [{ci:2d}] {cname:20s}: {count:6d} ({percentage:5.1f}%)")
    
    # 데이터 저장
    np.save(f"{CACHE_DIR}/X_train.npy", X_train)
    np.save(f"{CACHE_DIR}/y_train.npy", y_train)
    np.save(f"{CACHE_DIR}/X_test.npy", X_test)
    np.save(f"{CACHE_DIR}/y_test.npy", y_test)
    
    if X_val is not None:
        np.save(f"{CACHE_DIR}/X_val.npy", X_val)
        np.save(f"{CACHE_DIR}/y_val.npy", y_val)
    
    # 클래스 정보 저장
    with open(f"{CACHE_DIR}/classes.json", "w", encoding="utf-8") as f:
        json.dump(CLASSES, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 데이터 저장 완료!")
else:
    print("❌ 데이터를 로드할 수 없습니다. 폴더 구조를 확인해주세요.")


🚀 데이터 로드 시작...

📂 데이터 로드 중... (Train/Test 분리)

📁 TRAIN 폴더 처리 중...


/var/folders/9g/jc575gcs03ldzjf4y_2nmltm0000gn/T/ipykernel_9556/3475116199.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)


   [16] 정적                  :  16 파일 ✓
   [ 6] 소음_가구               :  35 파일 ✓
   [ 6] 소음_가구               :  35 파일 ✓
   [20] 폰_전화                :  15 파일 ✓
   [20] 폰_전화                :  15 파일 ✓
   [ 1] 개                   :  20 파일 ✓
   [ 1] 개                   :  20 파일 ✓
   [14] 안전_낙상               :  80 파일 ✓
   [ 3] 문_도어락               :  18 파일 ✓
   [14] 안전_낙상               :  80 파일 ✓
   [ 3] 문_도어락               :  18 파일 ✓
   [ 7] 소음_드럼               :  60 파일 ✓
   [ 9] 소음_아이발소리            :  70 파일 ✓
   [10] 소음_어른발소리            :  52 파일 ✓
   [18] 폰_링톤                :  33 파일 ✓
   [ 4] 문_초인종               :  22 파일 ✓
   [ 2] 문_노크                :  32 파일 ✓
   [ 0] 가전제품_종료             :  10 파일 ✓
   [12] 소음_통돌이              :  62 파일 ✓
   [ 5] 배경소음                :  27 파일 ✓
   [ 8] 소음_문소리              :  40 파일 ✓
   [17] 타이머_알람              :   7 파일 ✓
   [15] 안전_화재_경보            :  21 파일 ✓
   [19] 폰_알림                :  71 파일 ✓
   [13] 아기_울음               : 128 파일 ✓
   [11] 소음_청소기           

## 🧠 모델 학습 (EfficientNetB0 전이학습)

In [ ]:
from tensorflow.keras import layers, models

NUM_EPOCHS = 20
BATCH_SIZE = 32
LR = 1e-3
FINETUNE_AT = 100   # 상위 몇 개 레이어 미세조정

def load_cached_new(use_balanced=True, validation_source="test_folder"):
    """
    새로운 데이터 구조로 캐시된 데이터 로드 (3채널 변환 포함)
    
    Args:
        use_balanced: True면 균형 맞춘 Train 데이터 사용, False면 원본 사용
        validation_source: "test_folder" (test 폴더), "train_split" (train에서 20% 분할)
    """
    if use_balanced:
        print("✅ 균형 맞춘 Train 데이터셋 사용")
        Xtr = np.load(f"{CACHE_DIR}/X_train_balanced.npy")
        ytr = np.load(f"{CACHE_DIR}/y_train_balanced.npy")
    else:
        print("⚠️  원본 Train 데이터셋 사용 (불균형 가능)")
        Xtr = np.load(f"{CACHE_DIR}/X_train.npy")
        ytr = np.load(f"{CACHE_DIR}/y_train.npy")
    
    # 검증 데이터 선택
    if validation_source == "test_folder":
        try:
            Xval = np.load(f"{CACHE_DIR}/X_val.npy")
            yval = np.load(f"{CACHE_DIR}/y_val.npy")
            print("✅ Test 폴더를 검증 데이터로 사용")
        except:
            print("⚠️  Test 폴더가 없어서 Train 분할 데이터를 검증으로 사용")
            Xval = np.load(f"{CACHE_DIR}/X_test.npy")
            yval = np.load(f"{CACHE_DIR}/y_test.npy")
    else:
        Xval = np.load(f"{CACHE_DIR}/X_test.npy")
        yval = np.load(f"{CACHE_DIR}/y_test.npy")
        print("✅ Train 분할 데이터를 검증으로 사용")
    
    # 1채널 → 3채널 변환 (ImageNet 호환)
    print("🔄 1채널 → 3채널 변환 중...")
    if Xtr.shape[-1] == 1:
        Xtr = np.repeat(Xtr, 3, axis=-1)
    if Xval.shape[-1] == 1:
        Xval = np.repeat(Xval, 3, axis=-1)
    
    with open(f"{CACHE_DIR}/classes.json") as f:
        classes = json.load(f)
    
    print(f"Train: {Xtr.shape}, Val: {Xval.shape}")
    return (Xtr, ytr), (Xval, yval), classes

def build_model_simple(num_classes, input_shape=(128, 128, 3)):
    """
    간단하고 안정적인 CNN 모델 구성
    """
    inputs = layers.Input(shape=input_shape)
    
    # 단순한 CNN 구조
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.25)(x)
    
    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LR),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def build_model_efficientnet(num_classes, input_shape=(128, 128, 3)):
    """
    EfficientNet 기반 전이학습 모델 (가중치 없이)
    """
    inputs = layers.Input(shape=input_shape)
    
    # EfficientNetB0 백본 (가중치 없이)
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights=None,  # 가중치 없음
        input_tensor=inputs,
        pooling="avg"
    )
    
    x = base.output
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(LR),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# ==== 학습 실행 ====
print("=" * 60)
print("🧠 모델 학습 시작")
print("=" * 60)

# 균형 맞춘 데이터 + Test 폴더를 검증으로 사용
(Xtr, ytr), (Xval, yval), classes = load_cached_new(
    use_balanced=True, 
    validation_source="test_folder"
)

print(f"📚 클래스 수: {len(classes)}")
print(f"📊 Train 분포: {np.bincount(ytr)}")
print(f"📊 Val 분포: {np.bincount(yval)}")

# 데이터 정규화 (0-255 → 0-1)
print("🔄 데이터 정규화 중...")
Xtr = Xtr.astype(np.float32) / 255.0
Xval = Xval.astype(np.float32) / 255.0

# 모델 선택 (간단한 CNN으로 시작)
print("🏗️  모델 구성 중...")
try:
    # 먼저 간단한 CNN으로 시도
    model = build_model_simple(len(classes))
    model_type = "Simple CNN"
    print("✅ Simple CNN 모델 사용")
except Exception as e:
    print(f"❌ Simple CNN 실패: {e}")
    # 대안으로 EfficientNet (가중치 없음)
    model = build_model_efficientnet(len(classes))
    model_type = "EfficientNet (no weights)"
    print("✅ EfficientNet (가중치 없음) 모델 사용")

print(f"📝 모델 구조 ({model_type}):")
print(f"   입력: {Xtr.shape[1:]}")
print(f"   출력: {len(classes)} 클래스")
print(f"   총 파라미터: {model.count_params():,}")

# 콜백 설정
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        f"{MODEL_DIR}/best.keras",
        save_best_only=True, 
        monitor="val_accuracy"
    ),
    tf.keras.callbacks.EarlyStopping(
        patience=7,  # 더 많은 patience
        restore_best_weights=True, 
        monitor="val_accuracy"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        patience=3, 
        factor=0.5, 
        monitor="val_loss",
        min_lr=1e-7
    ),
]

# 학습 시작
print(f"\n? {model_type} 학습 시작...")
history = model.fit(
    Xtr, ytr, 
    validation_data=(Xval, yval),
    epochs=NUM_EPOCHS, 
    batch_size=BATCH_SIZE, 
    callbacks=callbacks, 
    verbose=1
)

# 최종 모델 저장
model.save(f"{MODEL_DIR}/soundlight_efficientnet.keras")
with open(f"{MODEL_DIR}/classes.json", "w", encoding="utf-8") as f:
    json.dump(classes, f, ensure_ascii=False, indent=2)

print(f"\n✅ 모델 저장 완료: {MODEL_DIR}/soundlight_efficientnet.keras")

# 최종 성능 평가
print(f"\n📊 최종 성능 ({model_type}):")
train_loss, train_acc = model.evaluate(Xtr, ytr, verbose=0)
val_loss, val_acc = model.evaluate(Xval, yval, verbose=0)
print(f"   Train: Loss={train_loss:.4f}, Accuracy={train_acc:.4f}")
print(f"   Val:   Loss={val_loss:.4f}, Accuracy={val_acc:.4f}")

# 성능 분석
if val_acc > 0.8:
    print(f"\n🎉 우수한 성능! ({val_acc:.3f})")
elif val_acc > 0.6:
    print(f"\n👍 괜찮은 성능 ({val_acc:.3f}). 추가 최적화 가능")
elif val_acc > 0.4:
    print(f"\n⚠️  중간 성능 ({val_acc:.3f}). 개선 필요:")
    print("   1. 더 많은 에포크 또는 데이터")
    print("   2. 모델 복잡도 조정")
    print("   3. 데이터 전처리 개선")
else:
    print(f"\n❌ 낮은 성능 ({val_acc:.3f}). 근본적 검토 필요:")
    print("   1. 데이터 품질 확인")
    print("   2. 레이블 정확성 검증")
    print("   3. 특성 추출 방법 재검토")
    print("   4. 클래스 균형 재조정")

# 학습 히스토리 요약
print(f"\n📈 학습 경과:")
print(f"   최고 Train Accuracy: {max(history.history['accuracy']):.4f}")
print(f"   최고 Val Accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"   최종 에포크: {len(history.history['accuracy'])}")

## 🔍 파일 추론

In [12]:

def specs_from_wav(path):
    y, sr = librosa.load(path, sr=SR, mono=True)
    win=int(SR*WIN_SEC); hop=int(SR*HOP_SEC)
    if len(y) < win: y = np.pad(y, (0, win-len(y)))
    specs = []
    for st in range(0, len(y)-win+1, hop):
        seg = y[st:st+win]
        mel = librosa.feature.melspectrogram(y=seg, sr=SR, n_fft=1024, hop_length=256,
                                             n_mels=N_MELS, fmin=FMIN, fmax=FMAX, power=2.0)
        logmel = librosa.power_to_db(mel, ref=np.max)
        zy = SPEC_SHAPE[0] / logmel.shape[0]
        zx = SPEC_SHAPE[1] / logmel.shape[1]
        img = zoom(logmel, (zy, zx), order=1)[..., None]
        specs.append(img.astype(np.float32))
    X = np.array(specs, dtype=np.float32)
    X = np.repeat(X, 3, axis=-1)
    return X

def infer_file(wav_path, model_path=f"{MODEL_DIR}/soundlight_efficientnet.keras"):
    m = tf.keras.models.load_model(model_path)
    with open(f"{MODEL_DIR}/classes.json") as f:
        classes = json.load(f)
    X = specs_from_wav(wav_path)
    probs = m.predict(X, verbose=0)
    mean_prob = probs.mean(axis=0)
    idx = int(mean_prob.argmax())
    cname = classes[idx]
    color = CLASS_TO_COLOR[cname]
    print(f"[FILE] {wav_path} → {cname} | color={color} | conf={mean_prob[idx]:.2f}")
    return cname, color, float(mean_prob[idx])

# 사용 예시:
# infer_file("sample.wav")


## 🎙 실시간 추론 (마이크)

In [13]:
import sounddevice as sd

# 공통 설정이 실행되지 않은 경우를 대비한 변수 정의
if 'SR' not in locals():
    print("⚠️  공통 설정이 실행되지 않았습니다. 셀 5번을 먼저 실행해주세요!")
    SR = 16000
    WIN_SEC = 1.0
    HOP_SEC = 0.5
    N_MELS = 64
    FMIN, FMAX = 50, 8000
    SPEC_SHAPE = (128, 128)
    MODEL_DIR = "../models"

EMA_ALPHA = 0.6   # 지수이동평균 스무딩
BLOCK = int(SR * HOP_SEC)  # 0.5초 블록

def _spec_from_sig(sig):
    mel = librosa.feature.melspectrogram(y=sig, sr=SR, n_fft=1024, hop_length=256,
                                         n_mels=N_MELS, fmin=FMIN, fmax=FMAX, power=2.0)
    logmel = librosa.power_to_db(mel, ref=np.max)
    zy = SPEC_SHAPE[0]/logmel.shape[0]; zx = SPEC_SHAPE[1]/logmel.shape[1]
    img = zoom(logmel, (zy, zx), order=1)[..., None]
    img = np.repeat(img, 3, axis=-1)[None, ...].astype(np.float32)
    return img

def infer_realtime(model_path=f"{MODEL_DIR}/soundlight_efficientnet.keras"):
    print("🎙️ 실시간 추론 시작...")
    print(f"모델 로드 중: {model_path}")
    
    try:
        m = tf.keras.models.load_model(model_path)
        print("✅ 모델 로드 성공!")
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return
    
    # 클래스와 색상 정보 확인
    if 'CLASSES' not in locals():
        print("❌ CLASSES가 정의되지 않았습니다. 셀 5번을 먼저 실행해주세요!")
        return
    
    print(f"📚 분류 클래스: {len(CLASSES)}개")
    print(f"   처음 5개: {', '.join(CLASSES[:5])}...")
    
    # 실시간 데이터 저장을 위한 큐와 상태
    audio_queue = queue.Queue(maxsize=20)
    buffer = np.zeros(int(SR * WIN_SEC))
    probs_smooth = np.zeros(len(CLASSES))
    
    def callback(indata, frames, time_info, status):
        if status:
            print(f"오디오 상태: {status}")
        audio_queue.put(indata.copy())
    
    print("📟 실시간 분류 시작! (Ctrl+C로 종료)")
    print("=" * 80)
    
    try:
        with sd.InputStream(samplerate=SR, channels=1, callback=callback, blocksize=BLOCK):
            while True:
                if not audio_queue.empty():
                    chunk = audio_queue.get().flatten()
                    buffer[:-len(chunk)] = buffer[len(chunk):]
                    buffer[-len(chunk):] = chunk
                    
                    # 스펙트로그램 생성 및 예측
                    spec = _spec_from_sig(buffer)
                    probs = m.predict(spec, verbose=0)[0]
                    probs_smooth = EMA_ALPHA * probs + (1-EMA_ALPHA) * probs_smooth
                    
                    # 최고 확률 클래스
                    pred_idx = np.argmax(probs_smooth)
                    pred_class = CLASSES[pred_idx]
                    pred_prob = probs_smooth[pred_idx]
                    pred_color = CLASS_TO_COLOR.get(pred_class, "GRAY")
                    
                    # 결과 출력
                    print(f"🔊 {pred_class:15s} | 확률: {pred_prob:.3f} | 색상: {pred_color}")
                    
                time.sleep(0.1)
    
    except KeyboardInterrupt:
        print("\n\n✅ 실시간 추론 종료")
    except Exception as e:
        print(f"\n\n❌ 에러 발생: {e}")
        import traceback
        traceback.print_exc()

# 사용 예시 (주석 처리):
# infer_realtime()

## ✅ 빠른 점검 (선택)

In [14]:

# 데이터/모델이 준비되었다면 아래를 차례로 호출해보세요.
# 1) 전처리 먼저 실행 → 2) 학습 → 3) 파일/실시간 추론

# ⚠️ 주의: 위 셀들을 먼저 순서대로 실행해야 합니다!
# 필요한 함수가 정의되지 않은 경우 아래 코드를 실행하세요:

# 파일 추론 예시 (모델 파일이 있는 경우):
# infer_file("path/to/audio.wav")

# 실시간 마이크 추론 (모델이 준비된 경우):
# infer_realtime()

print("✅ 셀 실행 완료!")
print("📌 사용 방법:")
print("   1. 셀 6 (🔍 파일 추론)부터 셀 7 (🎙 실시간 추론)까지 모두 실행하세요")
print("   2. 그 다음 이 셀을 실행하거나 직접 함수를 호출하세요")


✅ 셀 실행 완료!
📌 사용 방법:
   1. 셀 6 (🔍 파일 추론)부터 셀 7 (🎙 실시간 추론)까지 모두 실행하세요
   2. 그 다음 이 셀을 실행하거나 직접 함수를 호출하세요
